In [1]:
import os, sys

fp_repo = '/Users/rfouyang/workspace/services/ds-fintech'
if fp_repo not in sys.path:
    sys.path.insert(0, fp_repo)

In [28]:
from pathlib import Path
from pprint import pprint
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import toad

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from util import woe_helper
from util import woe_mono
from util import report_helper
from util import bm_helper
from util import lgbm_helper
from util import data_helper
from util import score_helper
from util import shap_helper

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# 加载数据

In [168]:
fp_base = Path('/Users', 'rfouyang', 'workspace')
#fp_base = Path('C:\\', 'Users', 'USER', 'workspace')
fp_user = Path(fp_base, 'data', 'bcard_data', 'user.csv')
fp_repayment = Path(fp_base, 'data', 'bcard_data', 'repayment.csv')

df_user = pd.read_csv(fp_user, index_col=None)
df_repayment = pd.read_csv(fp_repayment, index_col=None)

In [169]:
pk = 'user_id'
label = 'label'

In [170]:
df_data = df_user.merge(df_repayment, on=pk, how='left')

In [171]:
df_data['ur'] = df_data['spend'] / df_data['limit_amount']

In [172]:
df_data['pre_ur'] = df_data.sort_values("month_gap").groupby(pk)["ur"].shift(1)

In [173]:
df_data['ur_chg'] = df_data['ur'] / df_data['pre_ur']

In [174]:
t = 3
df_sub = df_data.loc[df_data['month_gap']<=t].copy(deep=True)

In [176]:
df_ft = df_sub.groupby(pk).agg(
    m1_cnt=('is_m1', 'sum'),
    m2_cnt=('is_m2', 'sum'),
    m3_cnt=('is_m3', 'sum')
)

In [177]:
def get_max_dpd(r):
    if r['is_m3'].sum()>0:
        return 3
    elif r['is_m2'].sum()>0:
        return 2
    elif r['is_m1'].sum()>0:
        return 1
    else:
        return 0

sr_max_dpd = df_sub.groupby(pk).apply(lambda r: get_max_dpd(r))

In [178]:
df_ft['max_dpd'] = sr_max_dpd

In [179]:
df_sub.groupby(pk).agg(
    ur_mean=('ur', 'mean'), 
    ur_max=('ur', 'max'),
    ur_increase_cnt=('ur_chg', lambda x: len([v for v in x if v>1]))
)

,ur_mean,ur_max,ur_increase_cnt
user_id,,,
000404dec593450b94368544532bb579,0.323397,0.813253,1
0004ce24722c4db0afa32127cce51992,0.459569,1.000000,0
00054c7380bd40faad587eeec5f9f435,0.385040,0.721704,1
00094b1ced704930b38717be0a659b8e,0.303497,0.396152,1
000a438726224f0d8ef92dd20a7aea65,0.309202,0.621351,1
...,...,...,...
fffbdfcea90b48faa2b1a3ba08539633,0.421562,0.815514,1
fffc30ddb7774c43a2226a9d5884dde3,0.458619,0.702433,1
fffc98c934ab454eb0d33b6edf7f3ae8,0.505633,0.881937,1


In [182]:
df_data[df_data['user_id']=='813ff6ff5d544c3e86665906cb67f15b']

,user_id,limit_amount,label,month_gap,pre_outstanding,outstanding,payment,spend,is_m1,is_m2,is_m3,ur,pre_ur,ur_chg
481692,813ff6ff5d544c3e86665906cb67f15b,160000,0,12,95641.94,55019.08,95642,55019.14,0,0,0,0.343870,0.417542,0.823556
481693,813ff6ff5d544c3e86665906cb67f15b,160000,0,11,55019.08,78282.86,43543,66806.78,0,0,0,0.417542,0.669311,0.623840
481694,813ff6ff5d544c3e86665906cb67f15b,160000,0,10,78282.86,185372.55,0,107089.69,1,0,0,0.669311,0.126616,5.286137
481695,813ff6ff5d544c3e86665906cb67f15b,160000,0,9,185372.55,31718.14,173913,20258.59,0,0,0,0.126616,0.588241,0.215245
481696,813ff6ff5d544c3e86665906cb67f15b,160000,0,8,31718.14,98456.71,27380,94118.57,0,0,0,0.588241,0.729020,0.806893
481697,813ff6ff5d544c3e86665906cb67f15b,160000,0,7,98456.71,123298.92,91801,116643.21,0,0,0,0.729020,0.000000,inf
481698,813ff6ff5d544c3e86665906cb67f15b,160000,0,6,123298.92,-0.08,123299,0.00,0,0,0,0.000000,0.545521,0.000000
481699,813ff6ff5d544c3e86665906cb67f15b,160000,0,5,-0.08,87283.33,0,87283.41,0,0,0,0.545521,0.416461,1.309897
481700,813ff6ff5d544c3e86665906cb67f15b,160000,0,4,87283.33,153917.15,0,66633.82,1,0,0,0.416461,0.000000,inf
481701,813ff6ff5d544c3e86665906cb67f15b,160000,0,3,153917.15,-0.85,153918,0.00,0,0,0,0.000000,0.505459,0.000000


In [184]:
df_data[df_data['user_id']=='d5e1413f60f3473296e60ecd26513480']

,user_id,limit_amount,label,month_gap,pre_outstanding,outstanding,payment,spend,is_m1,is_m2,is_m3,ur,pre_ur,ur_chg
96,d5e1413f60f3473296e60ecd26513480,80000,0,12,26136.94,46950.02,0,20813.08,1,0,0,0.260164,0.512128,0.508005
97,d5e1413f60f3473296e60ecd26513480,80000,0,11,46950.02,40969.26,46951,40970.24,0,0,0,0.512128,0.705583,0.725822
98,d5e1413f60f3473296e60ecd26513480,80000,0,10,40969.26,97415.90,0,56446.64,1,0,0,0.705583,0.000000,inf
99,d5e1413f60f3473296e60ecd26513480,80000,0,9,97415.90,-0.10,97416,0.00,0,0,0,0.000000,0.676771,0.000000
100,d5e1413f60f3473296e60ecd26513480,80000,0,8,-0.10,54141.58,0,54141.68,1,0,0,0.676771,0.368320,1.837453
101,d5e1413f60f3473296e60ecd26513480,80000,0,7,54141.58,83607.20,0,29465.62,1,1,0,0.368320,0.479876,0.767532
102,d5e1413f60f3473296e60ecd26513480,80000,0,6,83607.20,44413.30,77584,38390.10,0,0,0,0.479876,0.000000,inf
103,d5e1413f60f3473296e60ecd26513480,80000,0,5,44413.30,-0.70,44414,0.00,0,0,0,0.000000,0.221964,0.000000
104,d5e1413f60f3473296e60ecd26513480,80000,0,4,-0.70,17756.40,0,17757.10,0,0,0,0.221964,0.273461,0.811684
105,d5e1413f60f3473296e60ecd26513480,80000,0,3,17756.40,21876.27,17757,21876.87,0,0,0,0.273461,0.507050,0.539317


In [191]:
def get_payment_ratio(r):
    if r['pre_outstanding'] <=0:
        return 1
    elif r['pre_outstanding'] <=10:
        if r['payment'] == 0:
            return 0
        else:
            return 1
    else:
        return r['payment'] / r['pre_outstanding']

df_data['payment_ratio'] = df_data.apply(lambda r: get_payment_ratio(r), axis=1)

In [192]:
df_data

,user_id,limit_amount,label,month_gap,pre_outstanding,outstanding,payment,spend,is_m1,is_m2,is_m3,ur,pre_ur,ur_chg,payment_ratio
0,5f97c74f9eee425083c9535536c96f75,90000,0,12,25947.77,86565.62,0,60617.85,1,1,0,0.673532,0.293476,2.295014,0.000000
1,5f97c74f9eee425083c9535536c96f75,90000,0,11,86565.62,112978.46,0,26412.84,1,0,0,0.293476,0.411779,0.712702,0.000000
2,5f97c74f9eee425083c9535536c96f75,90000,0,10,112978.46,41926.58,108112,37060.12,0,0,0,0.411779,0.440470,0.934862,0.956926
3,5f97c74f9eee425083c9535536c96f75,90000,0,9,41926.58,81568.91,0,39642.33,1,0,0,0.440470,0.161623,2.725287,0.000000
4,5f97c74f9eee425083c9535536c96f75,90000,0,8,81568.91,40556.02,55559,14546.11,0,0,0,0.161623,0.001603,100.804643,0.681130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481699,813ff6ff5d544c3e86665906cb67f15b,160000,0,5,-0.08,87283.33,0,87283.41,0,0,0,0.545521,0.416461,1.309897,1.000000
481700,813ff6ff5d544c3e86665906cb67f15b,160000,0,4,87283.33,153917.15,0,66633.82,1,0,0,0.416461,0.000000,inf,0.000000
481701,813ff6ff5d544c3e86665906cb67f15b,160000,0,3,153917.15,-0.85,153918,0.00,0,0,0,0.000000,0.505459,0.000000,1.000006
481702,813ff6ff5d544c3e86665906cb67f15b,160000,0,2,-0.85,80872.61,0,80873.46,0,0,0,0.505459,0.501907,1.007078,1.000000


In [163]:
df_sample = pd.read_csv('/Users/rfouyang/workspace/data/bcard_data.csv', index_col=None)

In [164]:
lst_info = list()
for idx, row in df_sample.iterrows():
    for k in range(1, 13, 1):
        item = {
            'user_id': row['CUST_ID'],
            'month_gap': 13-k,
            'pre_outstanding': row[f"OS_{k-1}"],
            'outstanding': row[f"OS_{k}"],
            'payment': row[f"Payment_{k}"],
            'spend': row[f"Spend_{k}"],
            'is_m1': row[f"Delq1_{k}"],
            'is_m2': row[f"Delq2_{k}"],
            'is_m3': row[f"Delq3_{k}"]
        }
        lst_info.append(item)

In [165]:
df_repayment = pd.DataFrame(lst_info)
df_repayment.to_csv(Path(fp_base, 'data', 'bcard_data', 'repayment.csv'), index=None)

In [166]:
df_user = df_sample[['CUST_ID','Loan_Amount', 'label']].rename(columns={'CUST_ID': 'user_id',
                                                       'OS_0': 'cur_outstanding', 'Loan_Amount': 'limit_amount'})

In [167]:
df_user.to_csv(Path(fp_base, 'data', 'bcard_data', 'user.csv'), index=None)

In [181]:
df_data

,user_id,limit_amount,label,month_gap,pre_outstanding,outstanding,payment,spend,is_m1,is_m2,is_m3,ur,pre_ur,ur_chg
0,5f97c74f9eee425083c9535536c96f75,90000,0,12,25947.77,86565.62,0,60617.85,1,1,0,0.673532,0.293476,2.295014
1,5f97c74f9eee425083c9535536c96f75,90000,0,11,86565.62,112978.46,0,26412.84,1,0,0,0.293476,0.411779,0.712702
2,5f97c74f9eee425083c9535536c96f75,90000,0,10,112978.46,41926.58,108112,37060.12,0,0,0,0.411779,0.440470,0.934862
3,5f97c74f9eee425083c9535536c96f75,90000,0,9,41926.58,81568.91,0,39642.33,1,0,0,0.440470,0.161623,2.725287
4,5f97c74f9eee425083c9535536c96f75,90000,0,8,81568.91,40556.02,55559,14546.11,0,0,0,0.161623,0.001603,100.804643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481699,813ff6ff5d544c3e86665906cb67f15b,160000,0,5,-0.08,87283.33,0,87283.41,0,0,0,0.545521,0.416461,1.309897
481700,813ff6ff5d544c3e86665906cb67f15b,160000,0,4,87283.33,153917.15,0,66633.82,1,0,0,0.416461,0.000000,inf
481701,813ff6ff5d544c3e86665906cb67f15b,160000,0,3,153917.15,-0.85,153918,0.00,0,0,0,0.000000,0.505459,0.000000
481702,813ff6ff5d544c3e86665906cb67f15b,160000,0,2,-0.85,80872.61,0,80873.46,0,0,0,0.505459,0.501907,1.007078


In [162]:
112978.46-26412.84

86565.62000000001